<h1> GYM LOCATION </h1>
<h3>first we download population data from Tunisian National Institut of Statistics </h3>
<h4> the link : <a href='http://www.ins.nat.tn/'>http://www.ins.nat.tn/ </a></h4>
<h3>after that we read the data </h3>   

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import requests # library to handle requests
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bf3fbce9934b4b5eb2c79a5b6d99b278 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tOlx8rKUKa_6Cvw02Y946kgSvCm9VEkeiXfwNIjuN1zj',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_bf3fbce9934b4b5eb2c79a5b6d99b278.get_object(Bucket='capstoneproject-donotdelete-pr-qukq1xbxwbzoou',Key='population_tunis.xlsx')['Body']
body1 = client_bf3fbce9934b4b5eb2c79a5b6d99b278.get_object(Bucket='capstoneproject-donotdelete-pr-qukq1xbxwbzoou',Key='Tunisian_municipality.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
if not hasattr(body1, "__iter__"): body1.__iter__ = types.MethodType( __iter__, body1 )
    
    
CLIENT_ID = 'HGIAJWFJ1SFIEKZV404RF5PJMAT4OCHUSFJPERF4TG1TNBEU' # your Foursquare ID
CLIENT_SECRET = 'HNVFNQTOLDZKALTVB3OUZINTTUII1VFJQCRMZGIUD3TIORDO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30


df= pd.read_excel(body)
df.head()


In [ ]:
import io
import ibm_boto3

client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tOlx8rKUKa_6Cvw02Y946kgSvCm9VEkeiXfwNIjuN1zj',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

bytes_buffer = io.BytesIO()
client.download_fileobj(Bucket='capstoneproject-donotdelete-pr-qukq1xbxwbzoou', Key='Tunis.geojson', Fileobj=bytes_buffer)
byte_value = bytes_buffer.getvalue()
str_value = byte_value.decode()

import json

d = json.loads(str_value)
d

<h3>we use this data to find wich neighbourhood have the highest % population with age between 15 to 50 (gym visitor) </h3>
<h2>but first , we must clean the data </h2>

In [ ]:
#delete Unnamed row
df.iloc[0][0]='Neighborhood'
new_header = df.iloc[0]
df = df[1:]
for x in range (len(new_header)) :
    new_header[x]=new_header[x].replace('\n', "")
    new_header[x]=new_header[x].replace('ans', "year")


df.columns =new_header
df.head()

<h3>Select our target by age</h3>

In [ ]:
df1=df.iloc[: , [0, 4,5,6,7]]
df1.head()

In [ ]:
s=[0]
for i in range (len(df1)):
    s.append(df1.iloc[i,1:5].sum())

df1['age_value_sum']=pd.DataFrame(s)
df1.head()

<h3>now we search the coordinate of neighobrhoods </h3>

In [ ]:
import numpy as np
df1['latitude']=pd.Series([0] * 24)
df1['longitude']=pd.Series([0] * 24)
df1


In [ ]:
from geopy.geocoders import Nominatim
for i in range (1,len(df1)-1):
    
    address = df1['Neighborhood'][i]+',Tunis,Tunisia'
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    
    if location:
        df1['latitude'][i] = location.latitude
        df1['longitude'][i] = location.longitude
        
    else:
        df1['latitude'][i] = 'None'
        df1['longitude'][i] = 'None'
        print('we cant find the location of :',address)
        
        
        

In [ ]:
df1

<h3>we delete neighborhood who hasn't a location in this dataFrame</h3>
<h4 color ='red'>NB :if we use an other API or we collect more data may be we find the location of all neighborhood</h4>

In [ ]:
df1 = df1.replace(to_replace='None', value=np.nan).dropna()
df1.drop([22,23],inplace=True)

In [ ]:
df1.iloc[0, df1.columns.get_loc('latitude')] = 36.858154708278946
df1.iloc[0, df1.columns.get_loc('longitude')]=10.336781933624707
df1.iloc[1, df1.columns.get_loc('latitude')]=36.80761234097368
df1.iloc[1, df1.columns.get_loc('longitude')]=10.171900016252494
df1.iloc[2, df1.columns.get_loc('latitude')]=36.796856828491784
df1.iloc[2, df1.columns.get_loc('longitude')]=10.176148635332083
df1.iloc[3, df1.columns.get_loc('latitude')]=36.8044465832733
df1.iloc[3, df1.columns.get_loc('longitude')]=10.16555100837445
df1.iloc[4, df1.columns.get_loc('latitude')]=36.81616301530516
df1.iloc[4, df1.columns.get_loc('longitude')]=10.162096323163269
df1.head()

In [ ]:
df1=df1.sort_values(by=['age_value_sum'],ascending=False)
  



In [ ]:
!pip install folium
import folium
from folium import plugins
#state_geo = 'https://www.data4tunisia.org/s/resources/tunisian-map-shapefiles-1/20180421-172936/Tunisian_municipality.geojson'
m = folium.Map(location=[36, 10], zoom_start=7)

# Add the color for the chloropleth:
"""m.choropleth(
 geo_data=d,
 name='choropleth',
 data=df1,
 columns=['Neighborhood', 'age_value_sum'],
 key_on='properties.NAME_EN',
 fill_color='YlGn',
 fill_opacity=0.8,
 line_opacity=0.2,
 legend_name='population'
)
folium.LayerControl().add_to(m)"""


incidents = folium.plugins.MarkerCluster().add_to(m)
opc=0.8
i=0
clr='green'
for lat, lng, label in zip(df1.latitude, df1.longitude, df1.Neighborhood):
    i=i+1
    if i>6 :
        clr='red'
        opc=i*0.1-0.5
    else:
        opc=opc-0.05
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        color=clr,
        popup=label,
        fill = True,
        fill_color=clr,
        fill_opacity=opc
    ).add_to(m)

m


In [ ]:
latitude =36.807612
longitude =10.171900
search_query = 'gym'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [ ]:
import requests
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
latitude =36.847999
longitude =10.294356
search_query = 'gym'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1

In [ ]:

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(m)
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(m)
# display map
m

In [ ]:
x= folium.Map(location=[36, 10], zoom_start=7)
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(x)
# display map
x